In [1]:
pip install libcloud

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement libcloud (from versions: none)
ERROR: No matching distribution found for libcloud
Note: you may need to restart the kernel to use updated packages.


In [1]:
import mechanize
import numpy
import os
import queue
import random
import shutil
import socket
import string
import threading
import time
import urllib.request
from abc import ABC, abstractmethod
import pandas
import libcloud
import paramiko
from dataclasses import dataclass
from libcloud.compute.providers import get_driver
from libcloud.compute.types import Provider
from paramiko.buffered_pipe import PipeTimeout

In [2]:
SSH_USER = 'am72ghiassi'
### MARC ###
GCLOUD_ACCOUNT = '460631779290-compute@developer.gserviceaccount.com'
GCLOUD_KEY_PATH = 'quantative-2-34bd7bef0cbc.json'  # The path to the Service Account Key (a JSON file)
GCLOUD_PROJECT = '460631779290'  # GCloud project id

### ERWIN ###
#GCLOUD_ACCOUNT = '352392662801-compute@developer.gserviceaccount.com'
#GCLOUD_KEY_PATH = 'projectgroup-1-da6053635a93.json'
#GCLOUD_PROJECT = '352392662801'
###

DESIGN_CSV = 'design_final.csv'  # The CSV with the experiment design

In [3]:
ComputeEngine = get_driver(Provider.GCE)

driver = ComputeEngine(GCLOUD_ACCOUNT, GCLOUD_KEY_PATH, project=GCLOUD_PROJECT)
location = [l for l in driver.list_locations() if l.id == '2000'][0]
#network = [n for n in driver.ex_list_networks() if n.id ==
#           '6096184313360012863'][0]

try:
    # To prevent problems with GCloud reusing IP addresses
    os.remove(f'/home/{SSH_USER}/.ssh/known_hosts')
except:
    pass

ex_id = 'alqxmewb'#''.join(random.choice(string.ascii_lowercase)
        #        for i in range(8))  # Generate a random project id

EX_RUNTIME = 300  # seconds
print(f"Experiment ID: {ex_id}")


Experiment ID: alqxmewb


In [4]:


class Node(ABC):
    def __init__(self, driver, name, master=False, masterNode=None, index=0):
        self.driver = driver
        self.name = name
        self.machineType = 'n1-standard-1'
        if not master:
            if index % 2 == 0:
                self.machineType = 'n2-standard-4'
            else:
                self.machineType = 'c2-standard-4'
        
        if not master and masterNode == None:
            raise ValueError("Slave nodes need a master")
        self.master = masterNode
        
        self.node = [node for node in driver.list_nodes() if node.name == name][0]
        self.disk = [disk for disk in driver.list_volumes() if disk.name == name]
        
        #self.disk = self.driver.create_volume(40, f"boot-{self.name}", image='opendl-2', location=location)
        #self.node = self.driver.create_node(
         #   name, self.machineType, None, location=location, ex_boot_disk=self.disk)
        self.driver.wait_until_running([self.node])
        self.pubip = self.node.public_ips[0]
        self.privip = self.node.private_ips[0]
        self.connected = False

        for i in range(5):  # Try 5 times
            try:
                self.open_ssh()
                break
            except Exception as e:
                print(e)
                time.sleep(5)
        if not self.connected:
            raise RuntimeError(f"Can't connect to node {self.name}")
        self.start_type()

    def __del__(self):
        self.close_ssh()

    def open_ssh(self):
        self.ssh = paramiko.SSHClient()
        self.ssh.load_system_host_keys()
        self.ssh.set_missing_host_key_policy(paramiko.WarningPolicy())
        k = paramiko.RSAKey.from_private_key_file("/home/mdroogh/.ssh/gcp")
        print(self.pubip);
        self.ssh.connect(self.pubip, port=22, username=SSH_USER, pkey = k)
        self.connected = True

    def close_ssh(self):
        self.connected = False
        self.ssh.close()

    @abstractmethod
    def start_type(self):
        pass


@dataclass
class JobOptions:
    batch_size: int
    max_epochs: int


class MasterNode(Node):
    def start_type(self):
        stdin, stdout, stderr = self.ssh.exec_command(f'echo "SPARK_MASTER_HOST=\'{self.privip}\'" >> /home/{SSH_USER}/bd/spark/conf/spark-env.sh')
        if (len(stderr.read()) > 0):
            print(stdout.read())
            print(stderr.read())

        stdin, stdout, stderr = self.ssh.exec_command(
            '/home/{SSH_USER}/bd/spark/sbin/start-master.sh')
        if (len(stderr.read()) > 0):
            print(stdout.read())
            print(stderr.read())

    def submit(self, options: JobOptions):

        stdin, stdout, stderr = self.ssh.exec_command(f"/home/{SSH_USER}/bd/spark/bin/spark-submit --master {self.privip} --driver-cores 1 "
                                                      f"--driver-memory 1G --total-executor-cores 1 --executor-cores 1 --executor-memory 1G "
                                                      f"--py-files \"/home/{SSH_USER}/bd/spark/lib/bigdl-0.8.0-python-api,/home/{SSH_USER}/bd/codes/bi-rnn.py\" "
                                                      f"--properties-file \"/home/{SSH_USER}/bd/spark/conf/spark-bigdl.conf\" "
                                                      f"--jars \"/home/{SSH_USER}/bd/spark/lib/bigdl-SPARK_2.3-0.11.0-jar-with-dependencies.jar\" "
                                                      f"--conf \"spark.driver.extraClassPath=/home/{SSH_USER}/bd/spark/lib/bigdl-SPARK_2.3-0.11.0-jar-with-dependencies.jar\" "
                                                      f"--conf \"spark.executer.extraClassPath=bigdl-SPARK_2.3-0.11.0-jar-with-dependencies.jar /home/{SSH_USER}/bd/codes/bi-rnn.py\" "
                                                      f"--action train --dataPath \"/tmp/mnist\" --batchSize {options.batch_size} --endTriggerNum {options.max_epochs} "
                                                      f"--learningRate 0.01 --learningrateDecay 0.0002")
        if (len(stderr.read()) > 0):
            print(stdout.read())
            print(stderr.read())

    def cancel(self):
        br = mechanize.Browser()
        br.open(f"http://{self.pubip}:8080")

        def select_form(form):
            return form.attrs.get('action', None) == 'app/kill/'
        try:
            br.select_form(predicate=select_form)
        except mechanize._mechanize.FormNotFoundError:
            print("FormNotFoundError")
        except Exception as e:
            print("An error occurred during cancelloing.")
            print(e)
        br.submit()


class SlaveNode(Node):
    def start_type(self):
        stdin, stdout, stderr = self.ssh.exec_command(f'/home/{SSH_USER}/bd/spark/sbin/start-slave.sh spark://{self.master.privip}:7077')
        if (len(stderr.read()) > 0):
            print(stdout.read())
            print(stderr.read())

In [5]:
master = MasterNode(driver, f"master-{ex_id}-1", master=True)

35.194.2.213
b''
b''


In [6]:
experiments = pandas.read_csv(DESIGN_CSV)
#experiments.columns.values[0] = 'Index'
#experiments.set_index('Index')

if 'failure_rate' not in experiments:
    experiments['failure_rate'] = 0
if 'failure_duration' not in experiments:
    experiments['failure_duration'] = 0

print(experiments)
num_slaves = 2
print(f"Number of slaves: {num_slaves}")

os.makedirs(f"raw/{ex_id}", exist_ok=True)
shutil.copyfile(DESIGN_CSV, f"raw/{ex_id}/design20epochs.csv")

    Index  batch_size  max_epochs  learning_rate  memory_size      model  \
0       0         152           3          0.010        12000  lenet5.py   
1       1         152           3          0.010        12000  bi-rnn.py   
2       2         184           4          0.070        12000  lenet5.py   
3       3         184           4          0.070        12000  bi-rnn.py   
4       4         112          17          0.005        12000  lenet5.py   
..    ...         ...         ...            ...          ...        ...   
85     85         128          27          0.020        12000  bi-rnn.py   
86     86         192          18          0.010        12000  lenet5.py   
87     87         192          18          0.010        12000  bi-rnn.py   
88     88         256          12          0.015        12000  lenet5.py   
89     89         256          12          0.015        12000  bi-rnn.py   

    failure_rate  failure_duration  
0              0                 0  
1            

'raw/alqxmewb/design20epochs.csv'

In [17]:
slaves = [SlaveNode(driver, f"slave-{ex_id}-{i}", masterNode=master, index=i) for i in range(1, int(num_slaves)+1)]

146.148.83.103
34.122.7.130


In [ ]:
for idx, row in experiments.iterrows():
    print(f"Experiment {row['Index']}")
    options = JobOptions(int(row['batch_size']), 10)


    #for slave in slaves:
    #    slave.start_type()


    filename = f"{int(row.Index)}-nodes8-batch{row.batch_size}-epochs{row.max_epochs}-lrate{row.learning_rate}-memory{row.memory_size}-{row.model}.log"

    stdin, stdout, stderr = master.ssh.exec_command(f"/home/{SSH_USER}/bd/spark/bin/spark-submit --master spark://{master.privip}:7077 --driver-cores 1 "
                                                    f"--driver-memory 3G --total-executor-cores 8 --executor-cores 4 --executor-memory {int(row.memory_size)}M "
                                                    f"--py-files /home/{SSH_USER}/bd/spark/lib/bigdl-0.11.0-python-api.zip,/home/{SSH_USER}/bd/codes/{row.model} "
                                                    f"--properties-file /home/{SSH_USER}/bd/spark/conf/spark-bigdl.conf "
                                                    f"--jars /home/{SSH_USER}/bd/spark/lib/bigdl-SPARK_2.3-0.11.0-jar-with-dependencies.jar "
                                                    f"--conf spark.driver.extraClassPath=/home/{SSH_USER}/bd/spark/lib/bigdl-SPARK_2.3-0.11.0-jar-with-dependencies.jar "
                                                    f"--conf spark.executer.extraClassPath=bigdl-SPARK_2.3-0.11.0-jar-with-dependencies.jar /home/{SSH_USER}/bd/codes/{row.model} "
                                                    f"--action train --dataPath /tmp/mnist --batchSize {row.batch_size} --endTriggerNum {row.max_epochs} "
                                                    f"--learningRate {row.learning_rate} --learningrateDecay 0.0002 > {ex_id}-{filename}", timeout=(row.max_epochs * 72))



    try:
        print(stdout.read())
        print(stderr.read())
    except PipeTimeout:
        print("PipeTimeout")
        master = MasterNode(driver, f"master-{ex_id}-1", master=True)
    except socket.timeout:
        print("Socket timeout")
        master = MasterNode(driver, f"master-{ex_id}-1", master=True)
   # master.cancel()
    sftp = master.ssh.open_sftp()
    sftp.get(f'{ex_id}-{filename}', f'raw/{ex_id}/{filename}')

    #for slave in slaves:
           # slave.stop_failure_worker()

Experiment 0
b''
b''
Experiment 1
b''
b''
Experiment 2
b''
b''
Experiment 3
b''
b''
Experiment 4


In [25]:
for idx, row in experiments.iterrows():
    if row.Index == 160:



        #for slave in slaves:
        #    slave.start_type()


        filename = f"{int(row.Index)}-nodes8-batch{row.batch_size}-epochs{row.max_epochs}-lrate{row.learning_rate}-memory{row.memory_size}-{row.model}.log"
        sftp = master.ssh.open_sftp()
        sftp.get(f'{ex_id}-{filename}', f'raw/{ex_id}/{filename}')